# **CREDIT RISK MODELING - BUSINESS UNDERSTANDING**

**Problem:** Predict loan approval probability to minimize default risk

**Success Metrics:** Precision, Recall, AUC-ROC, Business KPIs (approval rate, expected loss)

**Target Variable:** Approved_Flag (P1=Low Risk, P2=Medium, P3=High, P4=Very High Risk)

**Business Context:** Financial institution loan approval decision support

## 🔎 Data Inspection, Cleaning, and Merging Overview

In this section, we prepare and validate the **Internal Bank Dataset**, **External CIBIL Dataset**, and the **Unseen Dataset** before merging them into a unified dataset for further modeling.  

The workflow ensures **data quality, consistency, and integrity** by checking merge keys, duplicates, missing values, and overlaps across datasets.

# **Data Loading and Initial Inspection**
This section loads the datasets and performs initial validation, including shape checks, duplicate identification, and sample data display.

In [23]:
import pandas as pd
import numpy as np
from pathlib import Path

**Load Datasets**
   - Load internal, external, and unseen CSV datasets into Pandas DataFrames.
   - Purpose: bring raw data into the Colab environment for inspection and processing.

In [24]:
# upload datasets (csv files) into colab environment

internal_df = pd.read_csv("../datasets/Internal_Bank_Dataset.csv")
external_df = pd.read_csv("../datasets/External_Cibil_Dataset.csv")
unseen_df = pd.read_csv("../datasets/Unseen_Dataset.csv")

# Dataset Link: https://www.kaggle.com/code/saurabhbadole/predictive-credit-risk-modeling

In [25]:
'''

# Convert Excel files to CSV while preserving all data
def excel_to_csv_safe(excel_filename, csv_filename):
    """
    Convert Excel to CSV without losing data
    """
    # Read Excel with all data preserved
    df = pd.read_excel(excel_filename,
                      keep_default_na=False,    # Preserve empty cells as empty strings
                      na_filter=False)          # Don't convert strings like 'NA' to NaN

    # Save to CSV with all data intact
    df.to_csv(csv_filename,
              index=False,           # Don't save row indices
              encoding='utf-8')      # Use UTF-8 encoding for special characters

    print(f"✅ Converted {excel_filename} → {csv_filename}")
    print(f"   Shape: {df.shape[0]} rows × {df.shape[1]} columns")
    return df

# Convert all three files
internal_df = excel_to_csv_safe("Internal_Bank_Dataset.xlsx", "Internal_Bank_Dataset.csv")
external_df = excel_to_csv_safe("External_Cibil_Dataset.xlsx", "External_Cibil_Dataset.csv")
unseen_df = excel_to_csv_safe("Unseen_Dataset.xlsx", "Unseen_Dataset.csv")

'''

'\n\n# Convert Excel files to CSV while preserving all data\ndef excel_to_csv_safe(excel_filename, csv_filename):\n    """\n    Convert Excel to CSV without losing data\n    """\n    # Read Excel with all data preserved\n    df = pd.read_excel(excel_filename,\n                      keep_default_na=False,    # Preserve empty cells as empty strings\n                      na_filter=False)          # Don\'t convert strings like \'NA\' to NaN\n\n    # Save to CSV with all data intact\n    df.to_csv(csv_filename,\n              index=False,           # Don\'t save row indices\n              encoding=\'utf-8\')      # Use UTF-8 encoding for special characters\n\n    print(f"✅ Converted {excel_filename} → {csv_filename}")\n    print(f"   Shape: {df.shape[0]} rows × {df.shape[1]} columns")\n    return df\n\n# Convert all three files\ninternal_df = excel_to_csv_safe("Internal_Bank_Dataset.xlsx", "Internal_Bank_Dataset.csv")\nexternal_df = excel_to_csv_safe("External_Cibil_Dataset.xlsx", "Externa

In [26]:
'''

# Verify the conversion preserved all data
def verify_conversion(excel_file, csv_file):
    """
    Verify that CSV contains the same data as Excel
    """
    excel_df = pd.read_excel(excel_file, keep_default_na=False, na_filter=False)
    csv_df = pd.read_csv(csv_file, keep_default_na=False, na_filter=False)

    print(f"📊 {excel_file} vs {csv_file}:")
    print(f"   Excel shape: {excel_df.shape}")
    print(f"   CSV shape: {csv_df.shape}")
    print(f"   Data identical: {excel_df.equals(csv_df)}")
    print()

# Verify all conversions
verify_conversion("Internal_Bank_Dataset.xlsx", "Internal_Bank_Dataset.csv")
verify_conversion("External_Cibil_Dataset.xlsx", "External_Cibil_Dataset.csv")
verify_conversion("Unseen_Dataset.xlsx", "Unseen_Dataset.csv")

'''

'\n\n# Verify the conversion preserved all data\ndef verify_conversion(excel_file, csv_file):\n    """\n    Verify that CSV contains the same data as Excel\n    """\n    excel_df = pd.read_excel(excel_file, keep_default_na=False, na_filter=False)\n    csv_df = pd.read_csv(csv_file, keep_default_na=False, na_filter=False)\n\n    print(f"📊 {excel_file} vs {csv_file}:")\n    print(f"   Excel shape: {excel_df.shape}")\n    print(f"   CSV shape: {csv_df.shape}")\n    print(f"   Data identical: {excel_df.equals(csv_df)}")\n    print()\n\n# Verify all conversions\nverify_conversion("Internal_Bank_Dataset.xlsx", "Internal_Bank_Dataset.csv")\nverify_conversion("External_Cibil_Dataset.xlsx", "External_Cibil_Dataset.csv")\nverify_conversion("Unseen_Dataset.xlsx", "Unseen_Dataset.csv")\n\n'

**Initial Validation & Sampling**

- Display 10 random rows from each dataset using .sample(10).

- Purpose: quick sanity check to verify file structure, values, and column alignment.

In [27]:
# display 10 random records for initial validation

print("Internal Bank Dataset:")
print(internal_df.sample(10), "\n")

print("External Cibil Dataset:")
print(external_df.sample(10), "\n")

print("Unseen Dataset:")
print(unseen_df.sample(10), "\n")

Internal Bank Dataset:
       PROSPECTID  Total_TL  Tot_Closed_TL  Tot_Active_TL  \
15246       15247         3              2              1   
18556       18557         1              1              0   
4626         4627         2              0              2   
41997       41998         2              0              2   
29626       29627         1              0              1   
40044       40045        10              5              5   
23764       23765         2              0              2   
31781       31782         6              3              3   
25268       25269         1              0              1   
2210         2211         1              1              0   

       Total_TL_opened_L6M  Tot_TL_closed_L6M  pct_tl_open_L6M  \
15246                    0                  0            0.000   
18556                    0                  0            0.000   
4626                     0                  0            0.000   
41997                    0               

# **Check Columns and Basic Dataset Info**

**Dataset Structure and Statistics**

- View column names and dataset metadata.

- Use .describe() for summary statistics and .info() for column types and null counts.

- Purpose: understand schema, distributions, and data types.

In [10]:
print("Internal columns:", internal_df.columns.tolist())
print("External columns:", external_df.columns.tolist())

print("Internal Dataset :-")
print(internal_df.describe())
print(internal_df.info())

print("External Dataset :-")
print(external_df.describe())
print(external_df.info())

Internal columns: ['PROSPECTID', 'Total_TL', 'Tot_Closed_TL', 'Tot_Active_TL', 'Total_TL_opened_L6M', 'Tot_TL_closed_L6M', 'pct_tl_open_L6M', 'pct_tl_closed_L6M', 'pct_active_tl', 'pct_closed_tl', 'Total_TL_opened_L12M', 'Tot_TL_closed_L12M', 'pct_tl_open_L12M', 'pct_tl_closed_L12M', 'Tot_Missed_Pmnt', 'Auto_TL', 'CC_TL', 'Consumer_TL', 'Gold_TL', 'Home_TL', 'PL_TL', 'Secured_TL', 'Unsecured_TL', 'Other_TL', 'Age_Oldest_TL', 'Age_Newest_TL']
External columns: ['PROSPECTID', 'time_since_recent_payment', 'time_since_first_deliquency', 'time_since_recent_deliquency', 'num_times_delinquent', 'max_delinquency_level', 'max_recent_level_of_deliq', 'num_deliq_6mts', 'num_deliq_12mts', 'num_deliq_6_12mts', 'max_deliq_6mts', 'max_deliq_12mts', 'num_times_30p_dpd', 'num_times_60p_dpd', 'num_std', 'num_std_6mts', 'num_std_12mts', 'num_sub', 'num_sub_6mts', 'num_sub_12mts', 'num_dbt', 'num_dbt_6mts', 'num_dbt_12mts', 'num_lss', 'num_lss_6mts', 'num_lss_12mts', 'recent_level_of_deliq', 'tot_enq', 'C

# **Define Data Paths and Merging Setup**
Set path and Recheck data after merging

In [11]:
# define merge key and paths to datasets

merge_key = "PROSPECTID"

internal_path = Path("Internal_Bank_Dataset.csv")
external_path = Path("External_Cibil_Dataset.csv")
unseen_path = Path("Unseen_Dataset.csv")
merged_save_path = Path("merged_internal_external.csv")

In [12]:
# Check if merge key exists in both datasets

if merge_key not in internal_df.columns or merge_key not in external_df.columns:
    raise KeyError(f"Merge key '{merge_key}' missing in one of the files.")

# **Dataset Shape and Duplicate Checks**

**Duplicate Detection in Merge Key**

- Detect duplicate PROSPECTIDs using .duplicated().sum().

- Print sample duplicate IDs if they exist.

- Purpose: prevents one-to-many merges that could corrupt the dataset.

In [13]:
# Dataset shapes
print("Shapes: internal:", internal_df.shape, "external:", external_df.shape, "unseen:", unseen_df.shape)

# Detect duplicates in merge key
int_dup_count = internal_df[merge_key].duplicated().sum()
ext_dup_count = external_df[merge_key].duplicated().sum()
print(f"Duplicate PROSPECTID count -> internal: {int_dup_count}, external: {ext_dup_count}")

Shapes: internal: (51336, 26) external: (51336, 62) unseen: (100, 42)
Duplicate PROSPECTID count -> internal: 0, external: 0


In [14]:
# Show sample duplicate PROSPECTID entries if exists

if int_dup_count:
    print("Sample duplicate PROSPECTIDs in internal:\n",
          internal_df[internal_df[merge_key].duplicated()][merge_key].unique()[:10])
if ext_dup_count:
    print("Sample duplicate PROSPECTIDs in external:\n",
          external_df[external_df[merge_key].duplicated()][merge_key].unique()[:10])

# **Data Cleaning and Merging**
Replace Sentinel Values (-99999 as Sentinel as seen from sample())

In [15]:
# replace sentinel with NaN for cleaning

SENTINEL = -99999

internal_df.replace(SENTINEL, np.nan, inplace=True)
external_df.replace(SENTINEL, np.nan, inplace=True)
unseen_df.replace(SENTINEL, np.nan, inplace=True)

**Inner Merge of Datasets**

- Merge internal_df and external_df on PROSPECTID.

- Use suffixes (_int, _ext) to differentiate overlapping columns.

- Purpose: combine datasets only for common IDs.

In [16]:
# perform inner merge
merged_df = pd.merge(internal_df, external_df, on=merge_key, how="inner", suffixes=("_int", "_ext"))

# output shape and sample columns
print("Merged (inner) shape:", merged_df.shape)
print("Merged columns example:", merged_df.columns.tolist()[:20])

# show sample data
print("Merged sample rows:")
print(merged_df.sample(10))

# save merged dataset
merged_df.to_csv(merged_save_path, index=False)
print("Merged dataset saved to:", merged_save_path)

Merged (inner) shape: (51336, 87)
Merged columns example: ['PROSPECTID', 'Total_TL', 'Tot_Closed_TL', 'Tot_Active_TL', 'Total_TL_opened_L6M', 'Tot_TL_closed_L6M', 'pct_tl_open_L6M', 'pct_tl_closed_L6M', 'pct_active_tl', 'pct_closed_tl', 'Total_TL_opened_L12M', 'Tot_TL_closed_L12M', 'pct_tl_open_L12M', 'pct_tl_closed_L12M', 'Tot_Missed_Pmnt', 'Auto_TL', 'CC_TL', 'Consumer_TL', 'Gold_TL', 'Home_TL']
Merged sample rows:
       PROSPECTID  Total_TL  Tot_Closed_TL  Tot_Active_TL  \
16669       16670         2              2              0   
29714       29715         1              0              1   
7182         7183         3              1              2   
3370         3371         2              0              2   
34484       34485         1              0              1   
15051       15052         6              3              3   
24177       24178         5              4              1   
20959       20960         2              0              2   
21559       21560         3   

**Merge Indicator for Outer Join Validation**

**Merge Indicator (Discrepancy Check)**

- Perform an outer merge with _merge indicator.

- Shows whether IDs exist in both datasets or only one.

- Purpose: identify mismatched records and data coverage.

In [17]:
# merge with indicator to check for discrepancies

indicator = pd.merge(internal_df[[merge_key]], external_df[[merge_key]], on=merge_key, how="outer", indicator=True)
print("Merge indicator counts (outer join):")
print(indicator['_merge'].value_counts().to_dict())

Merge indicator counts (outer join):
{'both': 51336, 'left_only': 0, 'right_only': 0}


# **Handling Unseen Dataset**


**Check PROSPECTID in Unseen Dataset**

 - If present, unseen can be merged later for predictions.

- If absent, it is feature-only → preprocessing pipeline must be applied separately.

- Purpose: prepare unseen data path for model inference.

In [18]:
# check PROSPECTID presence in unseen data

if 'PROSPECTID' in unseen_df.columns:
    print("Unseen dataset contains PROSPECTID. You can merge predictions on that key later.")
else:
    print("Unseen dataset does NOT contain PROSPECTID (contains only features). Keep unseen_df separate and apply same preprocessing pipeline before scoring.")

Unseen dataset does NOT contain PROSPECTID (contains only features). Keep unseen_df separate and apply same preprocessing pipeline before scoring.


✅ **Summary**

- Loaded raw datasets and inspected their structure.

- Validated merge key existence and dataset integrity.

- Handled duplicates and standardized missing values (NaN).

- Merged datasets safely using PROSPECTID.

- Checked discrepancies in coverage across internal and external IDs.

- Prepared unseen dataset path for downstream prediction.

This ensures a clean, consistent, and analysis-ready dataset for modeling.

# **Exploratory Data Analysis (EDA) & Visualization**
Load the merged dataset for analysis

In [19]:
df = pd.read_csv(merged_save_path)

*Basic Dataset Overview*
- Shape
- Columns & Data Types
- Target Distribution

**Load and Inspect Data**

- Purpose: Load the merged dataset and review its shape, column names, datatypes, and target class distribution.

- Why: Ensures we understand schema and check if the target variable (Approved_Flag) is balanced or imbalanced.

In [20]:
print("Shape:", df.shape)
print("\nColumns:\n", df.columns)
print("\nData types:\n", df.dtypes.value_counts())

print("\nTarget variable distribution (Approved_Flag):\n", df['Approved_Flag'].value_counts())

Shape: (51336, 87)

Columns:
 Index(['PROSPECTID', 'Total_TL', 'Tot_Closed_TL', 'Tot_Active_TL',
       'Total_TL_opened_L6M', 'Tot_TL_closed_L6M', 'pct_tl_open_L6M',
       'pct_tl_closed_L6M', 'pct_active_tl', 'pct_closed_tl',
       'Total_TL_opened_L12M', 'Tot_TL_closed_L12M', 'pct_tl_open_L12M',
       'pct_tl_closed_L12M', 'Tot_Missed_Pmnt', 'Auto_TL', 'CC_TL',
       'Consumer_TL', 'Gold_TL', 'Home_TL', 'PL_TL', 'Secured_TL',
       'Unsecured_TL', 'Other_TL', 'Age_Oldest_TL', 'Age_Newest_TL',
       'time_since_recent_payment', 'time_since_first_deliquency',
       'time_since_recent_deliquency', 'num_times_delinquent',
       'max_delinquency_level', 'max_recent_level_of_deliq', 'num_deliq_6mts',
       'num_deliq_12mts', 'num_deliq_6_12mts', 'max_deliq_6mts',
       'max_deliq_12mts', 'num_times_30p_dpd', 'num_times_60p_dpd', 'num_std',
       'num_std_6mts', 'num_std_12mts', 'num_sub', 'num_sub_6mts',
       'num_sub_12mts', 'num_dbt', 'num_dbt_6mts', 'num_dbt_12mts', 'num_l

# **Visualizations: Distributions and Correlation**

In [21]:
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# set global plot style
plt.style.use('ggplot')

ModuleNotFoundError: No module named 'matplotlib'

**Target Distribution Pie Chart**
- Purpose: Visualize class balance between approved vs rejected applications.

- Why: Class imbalance can strongly influence model performance and guide resampling strategies.

In [ ]:
# Target distribution pie chart

plt.figure(figsize=(6, 6))
target_counts = df['Approved_Flag'].value_counts()
plt.pie(target_counts.values, labels=target_counts.index, autopct='%1.1f%%')
plt.title('Approval Flag Distribution')

**Feature Distribution Histograms**
- Purpose: Plot frequency distributions of key numeric features.

- Why: Detect outliers, skewness, and common ranges in applicant characteristics.

How:

- Income is log-transformed to reduce skew.

- Each plot highlights different applicant attributes (creditworthiness, income, age, credit activity).

In [ ]:
# Credit Score Distribution Histogram

plt.figure(figsize=(8, 5))
plt.hist(df['Credit_Score'].dropna(), bins=50, color='skyblue', alpha=0.7)
plt.xlabel('Credit Score')
plt.ylabel('Frequency')
plt.title('Credit Score Distribution')

**Missing Values Visualization**
- Purpose: Identify top 20 columns with most missing values.

- Why: Guides data cleaning and feature engineering strategy.

In [ ]:
# Net Monthly Income Distribution (Log Scale)

plt.figure(figsize=(8, 5))
income_clean = df['NETMONTHLYINCOME'].dropna()
income_clean = income_clean[income_clean > 0]  # Filter positive values
plt.hist(np.log10(income_clean), bins=50, color='lightgreen', alpha=0.7)
plt.xlabel('Log10(Net Monthly Income)')
plt.ylabel('Frequency')
plt.title('Net Monthly Income Distribution (Log Scale)')

In [ ]:
# Age Distribution Histogram

plt.figure(figsize=(8, 5))
plt.hist(df['AGE'].dropna(), bins=50, color='orange', alpha=0.7)
plt.xlabel('Age')
plt.ylabel('Frequency')
plt.title('Age Distribution')

In [ ]:
# Total Trade Lines Histogram

plt.figure(figsize=(8, 5))
plt.hist(df['Total_TL'].dropna(), bins=50, color='purple', alpha=0.7)
plt.xlabel('Total Trade Lines')
plt.ylabel('Frequency')
plt.title('Total Trade Lines Distribution')

In [ ]:
# Missing Values Heatmap (Top 20 columns with most missing values)

missing_data = df.isnull().sum().sort_values(ascending=False)[:20]
plt.figure(figsize=(10, 6))
plt.barh(missing_data.index, missing_data.values, color='grey')
plt.xlabel('Number of Missing Values')
plt.title('Top 20 Columns with Missing Values')
plt.gca().invert_yaxis()

**Correlation Heatmap**
- Purpose: Examine pairwise correlations among numeric features.

- Why: Detect redundant features, multicollinearity, or strong predictive signals.

In [ ]:
# Correlation Matrix Heatmap for Numeric Features

numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
if 'PROSPECTID' in numeric_cols:
    numeric_cols.remove('PROSPECTID')

corr_matrix = df[numeric_cols].corr()

plt.figure(figsize=(16, 12))
mask = np.triu(np.ones_like(corr_matrix, dtype=bool))

sns.heatmap(
    corr_matrix,
    mask=mask,
    annot=False,
    cmap='coolwarm',
    center=0,
    square=True,
    fmt='.2f',
    cbar_kws={"shrink": .8}
)

plt.title('Correlation Matrix of Numeric Features', fontsize=18, pad=20)

plt.tight_layout()
plt.show()

**Box Plots by Approval Status**
- Purpose: Compare feature distributions across approval outcomes.

- Why: Highlights whether features like credit score, age, or income differ systematically between approved and rejected groups.

In [ ]:
# Box Plots for Key Features by Approval Status
fig, axes = plt.subplots(2, 2, figsize=(15, 10))
df.boxplot(column='Credit_Score', by='Approved_Flag', ax=axes[0, 0])
axes[0,0].set_title('Credit Score by Approval Status')
axes[0,0].set_xlabel('Approved_Flag')

df.boxplot(column='AGE', by='Approved_Flag', ax=axes[0, 1])
axes[0,1].set_title('Age by Approval Status')
axes[0,1].set_xlabel('Approved_Flag')

df.boxplot(column='Total_TL', by='Approved_Flag', ax=axes[1, 0])
axes[1,0].set_title('Total Trade Lines by Approval Status')
axes[1,0].set_xlabel('Approved_Flag')

# Log of Net Monthly Income
income_plot = df[df['NETMONTHLYINCOME'] > 0].copy()
income_plot['Log_Income'] = np.log10(income_plot['NETMONTHLYINCOME'])
income_plot.boxplot(column='Log_Income', by='Approved_Flag', ax=axes[1, 1])
axes[1,1].set_title('Log(Net Monthly Income) by Approval Status')
axes[1,1].set_xlabel('Approved_Flag')

plt.suptitle('')
plt.tight_layout()

# **Summary Statistics and Feature Analysis**
Providing descriptive stats for key variables stratified by target.

**Grouped Statistics**
- Purpose: Generate descriptive statistics (mean, median, std) of numeric features split by target.

- Why: Quantifies differences in applicant profiles across approval outcomes.

In [ ]:
# Key statistics by target
print("=== Credit Score Statistics per Approval Flag ===")
print(df.groupby('Approved_Flag')['Credit_Score'].describe())

print("\n=== Age Statistics per Approval Flag ===")
print(df.groupby('Approved_Flag')['AGE'].describe())

print("\n=== Total Trade Lines per Approval Flag ===")
print(df.groupby('Approved_Flag')['Total_TL'].describe())

'''
# Active-to-total trade line ratio
df['Active_to_Total_Ratio'] = df['Tot_Active_TL'] / (df['Total_TL'] + 1e-6)
print("\n=== Active to Total Trade Line Ratio per Approval Flag ===")
print(df.groupby('Approved_Flag')['Active_to_Total_Ratio'].describe())
'''

# **Categorical Variables Analysis**
Examine distribution across categories.

**Categorical Feature Analysis**
- Purpose: Analyze distributions of categorical features (e.g., gender, occupation, region) relative to approval flag.

- Why: Identifies categorical variables that may influence approval probability.

How:

- Crosstabs show counts and percentages.

- Stacked bar charts visualize category breakdown across approval outcomes.

In [ ]:
# Automatically detect categorical columns
categorical_cols = df.select_dtypes(include=['object', 'category']).columns.tolist()

# Exclude target column if present
if 'Approved_Flag' in categorical_cols:
    categorical_cols.remove('Approved_Flag')

for col in categorical_cols:
    if col in df.columns:
        print(f"\n=== {col} Distribution by Approval Flag ===")

        # Crosstab: counts
        counts = pd.crosstab(df[col], df['Approved_Flag'])

        # Crosstab: percentages
        percentages = pd.crosstab(df[col], df['Approved_Flag'], normalize='columns') * 100

        # Merge counts + percentages into one table
        summary = counts.astype(str) + " (" + percentages.round(1).astype(str) + "%)"
        print(summary)

        # Plot stacked bar chart
        ax = (percentages.T).plot(
            kind="bar",
            stacked=True,
            figsize=(8, 4),
            colormap="Set2"
        )
        plt.title(f"{col} Distribution by Approval Flag")
        plt.ylabel("Percentage")
        plt.xlabel("Approval Flag")

        # 🔑 Tilt x-axis labels
        plt.xticks(rotation=30, ha="right")

        plt.legend(title=col, bbox_to_anchor=(1.05, 1), loc="upper left")
        plt.tight_layout()
        plt.show()

# **Missing Values Analysis by Target**
Identify variables with missing data stratified by target class.

In [ ]:
missing_by_target = (
    df.drop(columns=['Approved_Flag'])
      .groupby(df['Approved_Flag'])
      .apply(lambda x: x.isnull().sum())
      .T
)

missing_by_target_pct = (
    df.drop(columns=['Approved_Flag'])
      .groupby(df['Approved_Flag'])
      .apply(lambda x: x.isnull().mean() * 100)
      .T
)

# sort the missingness across all target classes
top_missing = (
    missing_by_target_pct
    .assign(Max_Missing=lambda x: x.max(axis=1))   # new col = max missing %
    .sort_values(by='Max_Missing', ascending=False)
    .drop(columns=['Max_Missing'])
    .head(10)
)

In [ ]:
print("=== Missing Value Percentages by Approval Flag (Top 10 across all classes) ===")
display(top_missing.round(2))

# --- Visualization ---
ax = top_missing.plot(kind="bar", figsize=(12, 6), colormap="Set2")

plt.title("Top 10 Features with Missing Values by Approval Flag", fontsize=16, pad=15)
plt.ylabel("Missing %")
plt.xlabel("Feature")
plt.xticks(rotation=30, ha="right")

plt.gca().yaxis.set_major_formatter(plt.FuncFormatter(lambda y, _: f'{y:.0f}%'))
plt.legend(title="Approved_Flag", bbox_to_anchor=(1.05, 1), loc="upper left")

plt.tight_layout()

# **Data Quality Summary**
Summary statistics for dataset integrity.

In [ ]:
total_records = len(df)
total_features = len(df.columns)
numeric_features = len(numeric_cols)
categorical_features = len(categorical_cols)
missing_percentage = df.isnull().sum().sum() / (total_records * total_features) * 100

print(f"Total records: {total_records}")
print(f"Total features: {total_features}")
print(f"Numeric features: {numeric_features}")
print(f"Categorical features: {categorical_features}")
print(f"Overall missing value percentage: {missing_percentage:.2f}%")

In [ ]:
# Features with >50% missing data

high_missing = df.isnull().sum() / total_records * 100
high_missing_features = high_missing[high_missing > 50]

if not high_missing_features.empty:
    print(f"\nFeatures with >50% missing values ({len(high_missing_features)} features):")
    print(high_missing_features.sort_values(ascending=False))
else:
    print("\nNo features with >50% missing values found.")

# Handling Missing Values
Using Iterative Imputer for Bayesian Ridge Imputation

In [ ]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.preprocessing import LabelEncoder

In [ ]:
# make a new dataframe after handling missing values

df_reg = df.copy()

num_cols = df_reg.select_dtypes(include=[np.number]).columns
cat_cols = df_reg.select_dtypes(exclude=[np.number]).columns

In [ ]:
# encode categorical data for imputation

le_dict = {} # store imputations

for col in cat_cols:
    le = LabelEncoder()
    df_reg[col] = df_reg[col].astype(str)
    df_reg[col] = le.fit_transform(df_reg[col])
    le_dict[col] = le

In [ ]:
imputer = IterativeImputer(random_state=17, max_iter=10, sample_posterior=False) # check random value
df_reg[num_cols] = imputer.fit_transform(df_reg[num_cols])

In [ ]:
# decode transformed cols back to categorical

for col in cat_cols:
    df_reg[col] = le_dict[col].inverse_transform(df_reg[col].astype(int))

In [ ]:
# check imputation results

plt.figure(figsize=(14,6))
sns.heatmap(df_reg.isnull(), cbar=False, cmap="viridis")
plt.title("Missing Values Heatmap")

In [ ]:
df_reg.to_csv("merged_regression_imputed.csv", index=False)
print("Regression-imputed dataset saved as merged_regression_imputed.csv")

# Outlier Elimination
Based on BoxCox Plots and Percentile Clip

In [ ]:
from scipy.stats import boxcox

In [ ]:
# setup outlier detection from numerical columns

df_out = df_reg.copy()
num_cols = df_out.select_dtypes(include=[np.number]).columns

outlier_report = {}
transformation_report = {}

In [ ]:
for col in num_cols:

    Q1 = df_out[col].quantile(0.25)
    Q3 = df_out[col].quantile(0.75)

    IQR = Q3 - Q1
    lower = Q1 - 1.5 * IQR
    upper = Q3 + 1.5 * IQR

    outliers = ((df_out[col] < lower) | (df_out[col] > upper)).sum()
    pct_outliers = 100 * outliers / df_out.shape[0]
    outlier_report[col] = pct_outliers

In [ ]:
# show outlier percentage

outlier_df = pd.DataFrame.from_dict(outlier_report, orient='index', columns=['% Outliers'])
print("Outlier percentage per numeric column:\n", outlier_df.sort_values('% Outliers', ascending=False))

In [ ]:
for col in num_cols:
    skew = df_out[col].skew()
    pct_outliers = outlier_df.loc[col, '% Outliers']

    if pct_outliers > 5:  # Significant outliers
        if skew > 1:
            # Try Box-Cox (requires strictly positive values)
            if (df_out[col] > 0).all():
                df_out[col], _ = boxcox(df_out[col])  # Box-Cox transform
                transformation_report[col] = f"Box-Cox (skew={skew:.2f}, outliers={pct_outliers:.1f}%)"
            else:
                df_out[col] = np.log1p(df_out[col] - df_out[col].min() + 1)
                transformation_report[col] = f"log1p (skew={skew:.2f}, outliers={pct_outliers:.1f}%)"
        else:
            # Clip at 1st and 99th percentile
            Q1 = df_out[col].quantile(0.01)
            Q99 = df_out[col].quantile(0.99)
            df_out[col] = df_out[col].clip(Q1, Q99)
            transformation_report[col] = f"Clipped at 1st-99th pct (skew={skew:.2f}, outliers={pct_outliers:.1f}%)"
    else:
        transformation_report[col] = "No action (low outliers)"

In [ ]:
summary_df = pd.DataFrame.from_dict(transformation_report, orient='index', columns=['Action Taken'])
print("\nSummary of transformations:\n", summary_df)

In [ ]:
# Plot first 6 numerical columns

for col in num_cols[:6]:
    fig, axes = plt.subplots(1, 3, figsize=(15, 4))

    # Boxplots
    sns.boxplot(x=df_reg[col], ax=axes[0], color="skyblue")
    axes[0].set_title(f"Before: {col}")

    sns.boxplot(x=df_out[col], ax=axes[1], color="lightgreen")
    axes[1].set_title(f"After: {col}")

    # Distribution comparison
    sns.kdeplot(df_reg[col], ax=axes[2], label="Before", fill=True, alpha=0.4, color="blue")
    sns.kdeplot(df_out[col], ax=axes[2], label="After", fill=True, alpha=0.4, color="green")
    axes[2].set_title(f"Distribution: {col}")
    axes[2].legend()

    plt.tight_layout()
    plt.show()

# **Feature Selection**


In [ ]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.feature_selection import chi2, SelectKBest, f_regression, f_classif

In [ ]:
df_fs = df_out.copy()

In [ ]:
# select the target column

possible_targets = ["TARGET", "target", "label", "PerformanceTag", "Target"]
target = None

for col in df_fs.columns:
    if col in possible_targets:
        target = col
        break

if target is None:
    target = df_fs.columns[-1]

print(f"Target column detected: {target}")

In [ ]:
y = df_fs[target]
X = df_fs.drop(columns=[target])

In [ ]:
if y.dtype == "object":
    y = LabelEncoder().fit_transform(y)

In [ ]:
num_cols = X.select_dtypes(include=[np.number]).columns.tolist()
cat_cols = X.select_dtypes(include=['object', 'category']).columns.tolist()

In [ ]:
# categorical features encoding

if len(cat_cols) > 0:
    encoder = OneHotEncoder(drop="first", sparse_output=False, handle_unknown="ignore")
    X_cat = encoder.fit_transform(X[cat_cols])
    selected_cat_features = encoder.get_feature_names_out(cat_cols)
    X_cat_selected = X_cat
else:
    selected_cat_features = []
    X_cat_selected = np.empty((len(X),0))

In [ ]:
task = "classification"   # or "regression"
k = 20                    # number of top features to keep
force_keep = ["Credit_Score"]   # features you ALWAYS want to keep

In [ ]:
if len(num_cols) > 0:

    X_num = X[num_cols]
    scaler = StandardScaler()
    X_num_scaled = scaler.fit_transform(X_num)

    # Decide regression vs classification automatically
    if len(np.unique(y)) > 10:
        scores, p_values = f_regression(X_num_scaled, y)
    else:
        scores, p_values = f_classif(X_num_scaled, y)

    # Build DataFrame
    anova_df = pd.DataFrame({
        "Feature": num_cols,
        "F_Score": scores,
        "p_value": p_values
    })

    # Sort descending by F-score
    anova_df = anova_df.sort_values("F_Score", ascending=False).reset_index(drop=True)

    # Select top k features
    k = min(20, X_num.shape[1])
    top_features = anova_df.head(k)

    # Output
    print("\nANOVA/F-test scores (descending order):")
    print(anova_df)
    print(f"\nTop {k} numerical features (ANOVA/F-test):")
    print(top_features)

    # Keep top features
    selected_num_features = top_features["Feature"].values
    X_num_selected = X_num_scaled[:, [list(num_cols).index(f) for f in selected_num_features]]

else:
    selected_num_features = []
    X_num_selected = np.empty((len(X), 0))


In [ ]:
X_selected = np.hstack([X_cat_selected, X_num_selected])
selected_features = list(selected_cat_features) + list(selected_num_features)

df_selected = pd.DataFrame(X_selected, columns=selected_features, index=X.index)
df_selected[target] = y

print("\n Final selected dataset shape:", df_selected.shape)

In [ ]:
output_path = "selected_features.csv"
df_selected.to_csv(output_path, index=False)
print(f"Selected dataset saved to {output_path}")

# Selecting Common Features

In [ ]:
# training features (from your merged/cleaned dataset)
train_features = set(X.columns)

# unseen features (from your provided list)
unseen_features = set(unseen_df.columns)

# intersection
common_features = list(train_features.intersection(unseen_features))
print("Common features available for both training & unseen scoring:\n", common_features)

# keep only these for training
X_final = X[common_features]

# **Model Training**

In [ ]:
!pip install catboost

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import xgboost as xgb
from catboost import CatBoostClassifier

In [ ]:
# segregating columns for corresponding transformation

cat_cols = X_final.select_dtypes(include=["object"]).columns.tolist()
num_cols = X_final.select_dtypes(exclude=["object"]).columns.tolist()

In [ ]:
preprocessor = ColumnTransformer(
    transformers=[
        ("num", StandardScaler(), num_cols),
        ("cat", OneHotEncoder(handle_unknown="ignore", drop="first"), cat_cols),
    ]
)

In [ ]:
# split 80:20

y_final = y

X_train, X_val, y_train, y_val = train_test_split(
    X_final, y_final, test_size=0.2, random_state=17, stratify=y_final
)

In [ ]:
# list of models to be used and compared among

models = {
    "Logistic Regression": LogisticRegression(max_iter=1000),
    "Random Forest": RandomForestClassifier(n_estimators=200, random_state=17),
    "KNN": KNeighborsClassifier(n_neighbors=5),
    "Extra Trees": ExtraTreesClassifier(n_estimators=200, random_state=17),
    "Decision Tree": DecisionTreeClassifier(random_state=17),
    "Naive Bayes": GaussianNB(),
    "XGBoost": xgb.XGBClassifier( n_estimators=200, learning_rate=0.1, max_depth=7, random_state=17, eval_metric="logloss" ),
    "CatBoost": CatBoostClassifier( iterations=200, learning_rate=0.1, depth=7, random_state=17, verbose=0 )
}

In [ ]:
results = []

for name, model in models.items():

    pipe = Pipeline(steps=[("preprocessor", preprocessor), ("model", model)])
    pipe.fit(X_train, y_train)   # preprocess + fit model
    y_pred = pipe.predict(X_val) # preprocess + predict

    acc = accuracy_score(y_val, y_pred)
    prec = precision_score(y_val, y_pred, average="weighted", zero_division=0)
    rec = recall_score(y_val, y_pred, average="weighted")
    f1 = f1_score(y_val, y_pred, average="weighted")

    results.append([name, acc, prec, rec, f1])  # store results


In [ ]:
results_df = pd.DataFrame(
    results, columns=["Model", "Accuracy", "Precision", "Recall", "F1-score"]
)

print("\nModel Comparison Results:\n")
print(results_df.sort_values("F1-score", ascending=False))

In [ ]:
# visualization

plt.figure(figsize=(12, 6))
metrics = ["Accuracy", "Precision", "Recall", "F1-score"]

# Melt results for grouped bar chart
results_melted = results_df.melt(
    id_vars="Model",
    value_vars=metrics,
    var_name="Metric",
    value_name="Score"
)

# Sort models by F1-score before plotting
order = results_df.sort_values("F1-score", ascending=False)["Model"]

sns.barplot(
    data=results_melted,
    x="Model",
    y="Score",
    hue="Metric",
    order=order
)

plt.title("Model Performance Comparison", fontsize=14, weight="bold")
plt.ylabel("Score")
plt.xlabel("")
plt.ylim(0, 1.05)
plt.xticks(rotation=45, ha="right")
plt.legend(bbox_to_anchor=(1.05, 1), loc="upper left", title="Metric")
plt.tight_layout()
plt.show()
